In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import PIL
from PIL import Image
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
class DeepEmotion(nn.Module):
    def __init__(self, num_classes, regularization_lambda=0.001):
        super(DeepEmotion, self).__init__()

        self.regularization_lambda = regularization_lambda

        self.conv1 = nn.Conv2d(1, 10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 10, kernel_size=3)
        self.conv3 = nn.Conv2d(10, 10, kernel_size=3)
        self.conv4 = nn.Conv2d(10, 10, kernel_size=3)

        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.maxpool4 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(810, 50)
        self.fc2 = nn.Linear(50, num_classes)


        self.local_net = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, 2),
            nn.ReLU(True),

            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, 2),
            nn.ReLU(True),
        )

        self.local_fc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3*2),
        )

        self.local_fc[2].weight.data.zero_()
        self.local_fc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))


    def stn(self, x):
        xs = self.local_net(x)
        xs = xs.view(-1, 640)       # 10 * 3 * 3
        theta = self.local_fc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size(), align_corners=True)
        x = F.grid_sample(x, grid, align_corners=True)

        return x

    def calculate_regularization_loss(self):
        # Calculate the L2 regularization loss for the weights in the last two fully-connected layers
        regularization_loss = 0.0
        for param in self.fc1.parameters():
            regularization_loss += torch.norm(param, p=2)  # L2 norm
        for param in self.fc2.parameters():
            regularization_loss += torch.norm(param, p=2)  # L2 norm

        return regularization_loss

    def compute_loss(self, outputs, labels):
        # Cross-entropy loss
        classification_loss = nn.CrossEntropyLoss()(outputs, labels)

        # L2 regularization loss
        # regularization_loss = self.calculate_regularization_loss()

        # Total loss with regularization
        # total_loss = classification_loss + self.regularization_lambda * regularization_loss

        return classification_loss

    def forward(self, x):
        grid = self.stn(x)
        localization_grid_resized = F.interpolate(grid, size=(9, 9), mode='bilinear', align_corners=False)
        x = F.relu(self.conv1(x))
        x = self.conv2(x)
        x = F.relu(self.maxpool2(x))

        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        x = F.relu(self.maxpool4(x))

        x = F.dropout(x)

        x = x * localization_grid_resized

        x = x.view(-1, 810)
        x = self.fc1(x)
        x = self.fc2(x)

        # x = F.softmax(x, dim=1)

        return x

In [ ]:
class FERDataset(Dataset):
    def __init__(self, csv_data, transform, train=True):
        self.data = pd.read_csv(csv_data)
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if self.train:
            pixels = self.data.iloc[idx, 1].split()
            pixels = np.array(pixels, dtype=np.uint8).reshape(48, 48)

            image = Image.fromarray(pixels)

            label = int(self.data.iloc[idx, 0])

            if self.transform:
                image = self.transform(image)

            return image, label

        pixels = self.data.iloc[idx, 0].split()
        pixels = np.array(pixels, dtype=np.uint8).reshape(48, 48)

        image = Image.fromarray(pixels)

        if self.transform:
            image = self.transform(image)

        return image


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=0.5, std=0.5)
])

In [ ]:
def generate_TrainValTest_dataloaders(trpath='', tstpath='', batch_size=32):
    Train_fer_dataset = FERDataset(csv_data=trpath, transform=transform)

    train_size = int(0.8 * len(Train_fer_dataset))
    val_size = len(Train_fer_dataset) - train_size

    Train_fer_dataset, Val_fer_dataset = random_split(Train_fer_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    Test_fer_dataset = FERDataset(csv_data=tstpath, transform=transform, train=False)

    TrainDataLoader = DataLoader(Train_fer_dataset, batch_size=batch_size, shuffle=True)
    ValDataLoader = DataLoader(Val_fer_dataset, batch_size=batch_size)
    TestDataLoader = DataLoader(Test_fer_dataset, batch_size=batch_size)
    return TrainDataLoader, ValDataLoader, TestDataLoader

In [ ]:
emotion_dict = {
    0:'Angry',
    1:'Disgust',
    2:'Fear',
    3:'Happy',
    4:'Sad',
    5:'Surprise',
    6:'Neutral',
}

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# You can print the device to verify it's set to GPU
print("Device:", device)

Device: cuda


In [ ]:
def Train(epochs, trainloader, valloader, optimizer, model):
    print("===================================Start Training===================================")

    for epoch in range(epochs):
        train_loss = 0
        validation_loss = 0
        train_correct = 0
        val_correct = 0

        model.train()
        for data, labels in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{epochs} (Training)"):
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = model.compute_loss(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)


        model.eval()
        for data, labels in tqdm(valloader, desc=f"Epoch {epoch + 1}/{epochs} (Validation)"):
            data, labels = data.to(device), labels.to(device)
            val_outputs = model(data)

            val_loss = model.compute_loss(val_outputs, labels)

            validation_loss += val_loss.item()
            _, val_preds = torch.max(val_outputs, 1)
            val_correct += torch.sum(val_preds == labels.data)

        train_loss = train_loss / len(train_loader)
        train_acc = train_correct.double() / len(train_loader.dataset)
        validation_loss = validation_loss / len(val_loader)
        val_acc = val_correct.double() / len(val_loader.dataset)

        print('Epoch: {} \tTraining Loss: {:.8f} \tValidation Loss {:.8f} \tTraining Accuracy {:.3f}% \tValidation Accuracy {:.3f}%'
              .format(epoch + 1, train_loss, validation_loss, train_acc * 100, val_acc * 100))

    torch.save(model.state_dict(), 'deep_emotion-{}.pt'.format(epochs))
    print("===================================Training Finished===================================")



In [ ]:
num_classes = 7
epochs = 100
train_loader, val_loader, test_loader = generate_TrainValTest_dataloaders('/content/drive/MyDrive/FER_Data/train.csv', '/content/drive/MyDrive/FER_Data/test.csv')
model = DeepEmotion(num_classes=num_classes, regularization_lambda=0.001)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)


Train(epochs, train_loader, val_loader, optimizer, model)

===================================Start Training===================================


Epoch 1/100 (Validation): 100%|██████████| 180/180 [00:04<00:00, 42.47it/s]


Epoch: 1 	Training Loss: 1.78338051 	Validation Loss 1.72823739 	Training Accuracy 27.235% 	Validation Accuracy 30.773%


Epoch 2/100 (Validation): 100%|██████████| 180/180 [00:04<00:00, 43.21it/s]


Epoch: 2 	Training Loss: 1.72597072 	Validation Loss 1.70722755 	Training Accuracy 30.918% 	Validation Accuracy 31.400%


...

Epoch: 98 	Training Loss: 1.38807647 	Validation Loss 1.46971900 	Training Accuracy 46.841% 	Validation Accuracy 42.894%


Epoch 99/100 (Validation): 100%|██████████| 180/180 [00:03<00:00, 45.76it/s]


Epoch: 99 	Training Loss: 1.38494303 	Validation Loss 1.50598715 	Training Accuracy 47.133% 	Validation Accuracy 43.678%


Epoch 100/100 (Validation): 100%|██████████| 180/180 [00:04<00:00, 39.13it/s]

Epoch: 100 	Training Loss: 1.39250826 	Validation Loss 1.47132503 	Training Accuracy 46.732% 	Validation Accuracy 43.016%
===================================Training Finished===================================
